In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import re

import dask.dataframe as dd
import dask.array as da
# import dask.bag as db
import dask
from dask.distributed import Client, SSHCluster
import dask.distributed

# import hvplot.dask

In [ ]:
cluster.close()

In [2]:
cluster = SSHCluster(
    [ "bhbh-1", "bhbh-2", "bhbh-3", "bhbh-1","bhbh-2", "bhbh-3", "bhbh-1","bhbh-2", "bhbh-3", "bhbh-1","bhbh-2", "bhbh-3", "bhbh-1"],
    connect_options={"client_keys": "/home/ubuntu/private/tbertola_key.pem"},
    worker_options={"nthreads": 1}, # because each bhbh-* has 4 cores
    scheduler_options={"port": 8786, "dashboard_address": ":8787"}
)

2023-06-06 14:08:15,102 - distributed.deploy.ssh - INFO - 2023-06-06 14:08:15,099 - distributed.scheduler - INFO - State start
2023-06-06 14:08:15,109 - distributed.deploy.ssh - INFO - 2023-06-06 14:08:15,107 - distributed.scheduler - INFO -   Scheduler at:   tcp://10.67.22.140:8786
2023-06-06 14:08:16,265 - distributed.deploy.ssh - INFO - 2023-06-06 14:08:16,263 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:41623'
2023-06-06 14:08:16,270 - distributed.deploy.ssh - INFO - 2023-06-06 14:08:16,268 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:43255'
2023-06-06 14:08:16,401 - distributed.deploy.ssh - INFO - 2023-06-06 14:08:16,400 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:40045'
2023-06-06 14:08:16,544 - distributed.deploy.ssh - INFO - 2023-06-06 14:08:16,543 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.140:36631'
2023-06-06 14:08:16,743 - distributed.deploy.ssh - INFO - 2023-06-06

In [3]:
client = Client(cluster)

/home/ubuntu/miniconda3/lib/python3.10/site-packages/distributed/client.py:1386: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+------------------+
| Package | Client | Scheduler | Workers          |
+---------+--------+-----------+------------------+
| tornado | 6.3.2  | 6.3.2     | {'6.2', '6.3.2'} |
+---------+--------+-----------+------------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [4]:
client

<Client: 'tcp://10.67.22.140:8786' processes=12 threads=12, memory=23.29 GiB>

In [ ]:
# dask.config.set(scheduler='multiprocessing')

In [5]:
dask.config.get('scheduler')

'dask.distributed'

In [6]:
import glob
dir_path = '/mnt/bhbh/fiducial_Hrad_5M/sevn_output_*'
dir_list = glob.glob(dir_path)
print(len(dir_list))

60


In [7]:
output_column_to_remove = ['ID', 'MHE_0', 'MCO_0', 'Radius_0', 'RHE_0','RCO_0', 'Luminosity_0', 'Temperature_0', 'Lambda_0',
                               'Phase_0', 'PhaseBSE_0', 'Zams_0', 'MHE_1', 'MCO_1','Radius_1', 'RHE_1', 'RCO_1', 
                               'Luminosity_1', 'Temperature_1','Lambda_1', 'Phase_1', 'PhaseBSE_1', 'Zams_1']
    
evolved_column_to_remove = ['#ID', 'spin_0', 'Tstart_0', 'spin_1', 'Tstart_1', 'Tend', 'Dtout', 'Seed']
drop_list = ['RemnantType_0',  'RemnantType_1']

In [8]:
def FGpreprocessing(dir_path: str, n_thread: int, output_column_to_remove: list, evolved_column_to_remove: list, drop_list: list):
    
    output_str = f'{dir_path}/0/output_{n_thread}.csv'
    evolved_str = f'{dir_path}/0/evolved_{n_thread}.dat'
    logfile_str = f'{dir_path}/0/logfile_{n_thread}.dat'
    
#     Z_0 = float(re.findall(r".+((?<=Z)\d.\d+)", output_str)[0]) # in teoria non serve perchè Z_0 lo si prende negli evloved
    alpha = float(re.findall(r".+(?<=A)(.*)(?=L)", output_str)[0])
    ################################
    output = dd.read_csv(output_str).\
                rename(columns={'Mass_0':'Mass_0_out', 'Mass_1':'Mass_1_out'}).\
                drop(columns=output_column_to_remove)


    idxBHBH=(output.RemnantType_0==6) & (output.RemnantType_1==6) & (output.Semimajor.notnull())
    output=output[idxBHBH] #Distributed DataFrame containing only binaries BB

    ################################         

    evolved = dd.read_table(evolved_str, sep='\s+').\
                drop(columns=evolved_column_to_remove)
    evolved['alpha'] = alpha
    ################################

    logfile = dd.read_csv(logfile_str, header=None)

    df_RLO = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);RLO_BEGIN;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='RLO').\
                reset_index()

    df_CE = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);CE;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='CE').\
                reset_index()

    df_BSN = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);BSN;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='BSN').\
                reset_index()

    ##################################

    

    ################################

    bhbh = evolved.merge(output, on=['name'], how='inner').\
                   merge(df_RLO, on=['name'], how='left').\
                   merge(df_CE,  on=['name'], how='left').\
                   merge(df_BSN, on=['name'], how='left').\
                   fillna(value=0).\
                   drop(columns=drop_list)

    ###############################

    bhbh['tdelay'] = bhbh['GWtime'] + bhbh['BWorldtime']

    bhbh['Mass_max_out'] = bhbh['Mass_1_out']
    bhbh['Mass_max_out'] = bhbh['Mass_max_out'].\
                            where(cond=(bhbh['Mass_max_out'] > bhbh['Mass_0_out']), other=bhbh['Mass_0_out'])

    bhbh['q'] = bhbh['Mass_1_out']/bhbh['Mass_0_out']
    bhbh['q'] = bhbh['q'].\
                where(cond=(bhbh['Mass_1_out'] < bhbh['Mass_0_out']), other=bhbh['Mass_0_out']/bhbh['Mass_1_out'])

    bhbh['Mass_chirp'] = ((bhbh['Mass_0_out'] * bhbh['Mass_1_out'])**(3/5))/((bhbh['Mass_0_out'] + bhbh['Mass_1_out'])**(1/5))
    
    return bhbh

In [14]:
%%time
n_threads_DEMO = 30
bhbh_list=[]
for dir_name in dir_list[:5]:
    for i in range(n_threads_DEMO):
        _ = dask.delayed(FGpreprocessing)(dir_name, i,  output_column_to_remove, evolved_column_to_remove, drop_list)
        bhbh_list.append(_)

CPU times: user 49.5 ms, sys: 0 ns, total: 49.5 ms
Wall time: 48 ms


In [15]:
%%time
results = dask.compute(*bhbh_list) 

CPU times: user 1.12 s, sys: 110 ms, total: 1.23 s
Wall time: 8.72 s


In [16]:
%%time
results= dask.compute(*results)

CPU times: user 27.1 s, sys: 1.12 s, total: 28.2 s
Wall time: 23min 14s


In [17]:
cluster.close()

In [ ]:
list_results=list(results)

In [ ]:
type(list_results[0])

In [ ]:
res = dd.multi.concat(list_results)

In [ ]:
%%time
res_after_concat = res.compute()

In [ ]:
print(type(results))
print(results[0])

In [ ]:
%%time
res = dask.compute(*results) #return a list of pandas df

In [ ]:
%%time
results = dask.compute(*bhbh_list) 

In [ ]:
%%time
res = dd.from_delayed(bhbh_list) #return a list of pandas df

In [ ]:
type(res)

In [ ]:
test = res.compute()

In [ ]:
type(results[0])

In [ ]:
bhbh_test = bhbh.compute()

# bhbh_test.describe() # momentaneamente commentato dato che deve "solo" salvare in parquet

bhbh_test.to_parquet('/mnt/bhbh/parquet')

In [ ]:
# bhbh_list=[]
# for dir_name in dir_list:
#     bhbh_list.append(preprocessing(dir_name, output_column_to_remove, evolved_column_to_remove))   

In [ ]:
# bhbh = dd.multi.concat(bhbh_list)

In [ ]:
# test = bhbh.compute()
# test.head()